In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
import math

In [2]:
if(True):
    df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')
    df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
    df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
    df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

    df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
    df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
    df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

    df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
    df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
    df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

    param = [ \
              'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
              'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
              'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
              'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
              'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
              'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
             ]

    Oorsprongdata,rest = df_cleaned['Oorsprong'].str.split(".").str
    df_cleaned['Categorie'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = Oorsprongdata.str.split("_").str
    df_cleaned['Categorie'] = [ int(x[3:]) for x in df_cleaned['Categorie']] 
    df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
    df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
    df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
    # geef elke patient een uniek nummer
    df_cleaned['pat'] = df_cleaned['Categorie']*1000+df_cleaned['pat']

    df = df_cleaned[df_cleaned.index % 5 == 0]

    yValueColumn = 'Categorie'

    X_param = np.matrix(df_cleaned[param])
    y = df_cleaned[yValueColumn]

In [3]:
if(False):
    df_cleaned = pd.read_csv('all_predictions.csv', sep=';')
    param = ['Cat1', 'Cat2', 'Cat3', 'Cat4']
    df_cleaned[['Cat1', 'Cat2', 'Cat3', 'Cat4']] = df_cleaned[['Cat1', 'Cat2', 'Cat3', 'Cat4']].stack().str.replace(',','.').unstack()
    yValueColumn = 'groundtruth'
    X_param = np.matrix(df_cleaned[param])
    y = df_cleaned[yValueColumn]
    print(df_cleaned.head())

In [4]:
# Use SKLearn.linear_model
model_param = LinearRegression()
model_param.fit(X_param, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [5]:
from sklearn.metrics import mean_squared_error

y_pred = model_param.predict(X_param)
mean_squared_error(y_pred, y)

0.404627790233367

In [6]:
# kolom met voorspelling toevoegen aan data)
df_cleaned['predict'] = y_pred
df_cleaned['predict_round'] = [int(round(i)) if i <4 and i > 1 else 4 if i > 4 else 1 for i in y_pred]

In [7]:
## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

# aantal false tellen
df2 = df_cleaned.groupby(['pat','predict_round']).size().to_frame('countF').reset_index()
df2.set_index(['pat', 'predict_round'], inplace=True)
df2 = df2.unstack(-1)
df2 = df2.join(df_cleaned[['pat',yValueColumn]].drop_duplicates(subset=['pat', yValueColumn]).set_index('pat'))

def CalculatePerc(cat, CountF1, CountF2, CountF3, CountF4):
    Count = [CountF1, CountF2, CountF3, CountF4]
    perc = 100*Count[cat-1]/np.nansum(Count)
    if(math.isnan(perc)):
        perc = 0
    return perc
#  = 100*df4['countT'] / (df4['countT'] + df4['countF'])
df2['percentage'] = np.vectorize(CalculatePerc)(df2[yValueColumn], df2['countF', 1],df2['countF', 2],df2['countF', 3],df2['countF', 4])
df2.sort_values(by=['percentage'],ascending=False)
# df2.to_csv('test.csv', sep=';')
# print(df2)

# # df2 = df2.where(df2['predict_round'] != df2['Categorie']).dropna()
# # df2 = df2.set_index('pat')

# # aantal true tellen
# df3 = df_cleaned.groupby(['pat','predict_round']).size().to_frame('countT').reset_index()
# print(df3)
# df3 = df3.where(df2['predict_round'] == df2['Categorie']).dropna()
# df3 = df3.set_index('pat')

# # join df2 en df3
# df4 = pd.concat([df2, df3], axis=1)
# df4.drop(['predict_round','predict_round'], axis=1, inplace=True)
# df4.fillna(0, inplace=True)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(countF, 1)","(countF, 2)","(countF, 3)","(countF, 4)",Categorie,percentage
pat,,,,,,
2039,NaN,864.0,1.0,NaN,2,99.884393
2037,NaN,1565.0,2.0,NaN,2,99.872368
1021,820.0,32.0,NaN,NaN,1,96.244131
2001,NaN,1487.0,90.0,NaN,2,94.292961
2036,4.0,1468.0,105.0,NaN,2,93.088142
2038,NaN,946.0,88.0,NaN,2,91.489362
2026,32.0,1893.0,155.0,NaN,2,91.009615
2010,133.0,1934.0,61.0,NaN,2,90.883459
3031,NaN,89.0,1898.0,140.0,3,89.233662


In [8]:
ParamCheck = pd.DataFrame()
ParamCheck['Params'] = param
ParamCheck['Theta'] = list(model_param.coef_)
ParamCheck['Absolute'] = np.absolute(list(ParamCheck['Theta']))
ParamCheck = ParamCheck.sort_values(by=['Absolute'])
print(ParamCheck)

             Params     Theta  Absolute
11    clavicula_r_z -0.000026  0.000026
7     clavicula_l_y -0.001619  0.001619
21    scapula_x_dif  0.001775  0.001775
24    humerus_x_dif -0.002213  0.002213
3       humerus_r_x  0.002580  0.002580
10    clavicula_r_y -0.003479  0.003479
23    scapula_z_dif -0.003682  0.003682
26    humerus_z_dif  0.004249  0.004249
2       humerus_l_z -0.004285  0.004285
18  clavicula_x_dif  0.004573  0.004573
14      scapula_l_z  0.005046  0.005046
0       humerus_l_x -0.005364  0.005364
1       humerus_l_y -0.005549  0.005549
5       humerus_r_z  0.005674  0.005674
4       humerus_r_y -0.006017  0.006017
25    humerus_y_dif  0.006619  0.006619
19  clavicula_y_dif  0.006926  0.006926
8     clavicula_l_z -0.007066  0.007066
17      scapula_r_z  0.007191  0.007191
22    scapula_y_dif -0.007565  0.007565
9     clavicula_r_x -0.008378  0.008378
15      scapula_r_x -0.008776  0.008776
16      scapula_r_y  0.009182  0.009182
6     clavicula_l_x  0.010079  0.010079
